## StarSpace

StarSpace [[1]](#fn1) is an entity embedding approach which uses a similarity function between entities to construct a prediction task for a neural network. It maps objects of different types into a common vector space where they can be compared to each other. StarSpace can learn word, sentence and document level embeddings, ranking, text classification, embedding graphs, image classification, etc.

This notebook requires a working SparSpace program which can be built on any modern Linux or Windows machine as described in the building instructions in the [GitHub repository](https://github.com/facebookresearch/StarSpace). Additional packages required are:

- gensim==3.8.3
- matplotlib==3.3.2
- scikit-learn==0.23.2

-----
<span id="fn1"> [1] Ledell Yu Wu, Adam Fisch, Sumit Chopra, Keith Adams, Antoine Bordes, and Jason Weston. Starspace: Embed all the things! In Proceedings of the 32nd AAAI Conference on Artificial Intelligence, pages 5569–5577, 2018. </span>

----

We follow the official documentation of StarSpace and present the text classification example. First, we compile the starspace binary.

In [1]:
!git clone git@github.com:facebookresearch/StarSpace.git

Cloning into 'StarSpace'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 873 (delta 0), reused 0 (delta 0), pack-reused 868
Receiving objects: 100% (873/873), 3.05 MiB | 5.35 MiB/s, done.
Resolving deltas: 100% (567/567), done.


In [2]:
!cd StarSpace && make

g++ -pthread -std=gnu++11 -O3 -funroll-loops -g -c src/utils/normalize.cpp
g++ -pthread -std=gnu++11 -O3 -funroll-loops -I/usr/local/bin/boost_1_63_0/ -g -c src/dict.cpp
In file included from src/dict.cpp:8:0:
src/dict.h:22:10: fatal error: boost/format.hpp: No such file or directory
 #include <boost/format.hpp>
          ^~~~~~~~~~~~~~~~~~
compilation terminated.
makefile:58: recipe for target 'dict.o' failed
make: *** [dict.o] Error 1


The executable is now available as `data/StarSpace/starspace`. The original bash script for the text classification example is available in the [Starspace GitHub repository](https://github.com/facebookresearch/Starspace/blob/master/examples/classification_ag_news.sh). We reimplement it as a Jupyter notebook.

The data is based on [Antonio Gulli's corpus (AG)](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) which is a collection of more than 1 million news articles. From this collection, Zhang et al. [[2]](#fn2) constructed a smaller corpus, containing only the four largest news categoriess from the original corpus. Each category (i.e. class value) contains 30,000 training instances and 1,900 testing instances. The total number of training samples is 120,000 and 7,600 samples are resrved for testing. We download, unpack and inspect the corpus.

----
<span id="fn2"> [2] Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks for Text Classification. Advances in Neural Information Processing Systems 28 (NIPS 2015).</span>

In [3]:
!wget -c https://dl.fbaipublicfiles.com/starspace/ag_news_csv.tar.gz -P data
!cd data && tar -xzvf ag_news_csv.tar.gz
!ls data

--2020-11-14 19:23:52--  https://dl.fbaipublicfiles.com/starspace/ag_news_csv.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 2606:4700:10::ac43:904, 2606:4700:10::6816:4b8e, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|2606:4700:10::ac43:904|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11784327 (11M) [application/gzip]
Saving to: ‘data/ag_news_csv.tar.gz’

ag_news_csv.tar.gz  100%[===================>]  11.24M  5.71MB/s    in 2.0s    

2020-11-14 19:23:55 (5.71 MB/s) - ‘data/ag_news_csv.tar.gz’ saved [11784327/11784327]

ag_news_csv/
ag_news_csv/train.csv
ag_news_csv/test.csv
ag_news_csv/classes.txt
ag_news_csv/readme.txt
ag_news_csv  ag_news_csv.tar.gz


There are four classes and each news from the train and test set is classified using the line number of the actual class value.

In [5]:
!cat data/ag_news_csv/classes.txt
!head -n 5 data/ag_news_csv/train.csv

World
Sports
Business
Sci/Tech
"3","Wall St. Bears Claw Back Into the Black (Reuters)","Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
"3","Carlyle Looks Toward Commercial Aerospace (Reuters)","Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market."
"3","Oil and Economy Cloud Stocks' Outlook (Reuters)","Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums."
"3","Iraq Halts Oil Exports from Main Southern Pipeline (Reuters)","Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday."
"3","Oil prices soar to all-time re

We read the data into a Pandas DataFrame object and preprocess the text by converting it to lowercase and replacing a number of characters. The category is prefixed with `__label__` as required for the fastText word embedding file format. The transformed data is randomly shuffled and written into a fastText compatible text file. The train and test data are balanced concerning the four categories.

In [6]:
import pandas as pd
import os
from pprint import pprint

idx2category = {1: '__label__world',2: '__label__sports', 3:'__label__business', 4:'__label__scitech'}

def preprocess(df):
    df = df.replace({'category': idx2category})
    df['text'] = df['title'] + ' ' + df['body']
    df = df.drop(labels=['title', 'body'], axis=1)
    df['text'] = df['text'].str.lower()
    for s, rep in [("'"," ' "),
                   ('"',''),
                   ('.',' . '),
                   ('<br />',' '),
                   (',',' , '),
                   ('(',' ( '),
                   (')',' ) '),
                   ('!',' ! '),
                   ('?',' ? '),
                   (';',' '),
                   (':',' '),
                   ('\\',''),
                   ('  ',' ')
                  ]:
        df['text'] = df['text'].str.replace(s, rep)   
    df = df.sample(frac=1)
    return df

for filename in ['data/ag_news_csv/train.csv','data/ag_news_csv/test.csv']:
    df = pd.read_csv(filename, names=['category', 'title', 'body'])
    df = preprocess(df)
    print('File {}'.format(os.path.split(filename)[1]))
    pprint(df['category'].value_counts().to_dict())
    with open('{}.pp'.format(os.path.splitext(filename)[0]), 'w') as fp:
        for row in df.itertuples():
            fp.write('{} {}\n'.format(row.category, row.text))

File train.csv
{'__label__business': 30000,
 '__label__scitech': 30000,
 '__label__sports': 30000,
 '__label__world': 30000}
File test.csv
{'__label__business': 1900,
 '__label__scitech': 1900,
 '__label__sports': 1900,
 '__label__world': 1900}


We can now run StarSpace on the preprocessed files. The set of parameters is the same as in the example from the StarSpace repository. The `trainMode=0` and `fileFormat='FastText'` combinations defines the mode where the labels are individual words, i.e. the classification task. 

In [7]:
!./StarSpace/starspace train \
  -trainFile "data/ag_news_csv/train.pp" \
  -model "data/ag_news_csv/model" \
  -initRandSd 0.01 \
  -adagrad false \
  -ngrams 1 \
  -lr 0.01 \
  -epoch 5 \
  -thread 20 \
  -dim 10 \
  -negSearchLimit 5 \
  -trainMode 0 \
  -label "__label__" \
  -similarity "dot" \
  -verbose false

/bin/bash: ./StarSpace/starspace: No such file or directory


The resulting Starspace model embeddsthe input into a common 10-dimensional space (set by the `-dim 10` setting). We load it into a dataframe and inspect it. As shown in the table below, the model embedds everything into a common space: words that are present in documents but also the categories (the last four rows). In this way, we can now compare entities of different kinds.

In [8]:
pd.read_csv('data/ag_news_csv/model.tsv', sep='\t', header=None, keep_default_na=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/ag_news_csv/model.tsv'

Wen compute predictions and measure the peformance. In the test mode, StarSpace reports the hit@k evaluation metric which tells us how many correct answers are among the top k predictions. We are interested in the most probable category, therefore we use the hit@1 metric (in general, assignment of categories to text can be viewed as a multi-label classification problem). StarSpace achieves the score $hit@1=0.46$ which means that in 46% of test cases the model's first prediction is the correct answer.

In [9]:
!./StarSpace/starspace test \
  -model "data/ag_news_csv/model" \
  -testFile "data/ag_news_csv/test.pp" \
  -ngrams 1 \
  -dim 10 \
  -label "__label__" \
  -thread 10 \
  -similarity "dot" \
  -trainMode 0 \
  -verbose false \
  -predictionFile "data/ag_news_csv/test.y"

/bin/bash: ./StarSpace/starspace: No such file or directory


The performance in this example is not good and differs significantly from the published results [[1]](#fn1) where the authors report 91.6% accuracy on the test set for this task. Is is unclear, what is the reason for this discrepancy. Demonstrating the performance of a baseline classifier based on TF-IDF + SVM shows similar performance of the BOW + multinomial logistic regression reported in the paper.

In [10]:
import gensim
def to_tfidf(documents, dic=None, tfidf_model=None):
    documents = [gensim.parsing.preprocessing.preprocess_string(doc) for doc in documents]
    if dic is None:
        dic = gensim.corpora.Dictionary(documents)
        dic.filter_extremes()
    bows = [dic.doc2bow(doc) for doc in documents]
    if tfidf_model is None:
        tfidf_model = gensim.models.tfidfmodel.TfidfModel(dictionary=dic)
    tfidf_vectors = tfidf_model[bows]
    return tfidf_vectors, dic, tfidf_model


train = pd.read_csv('data/ag_news_csv/train.csv', names=['category', 'title', 'body'])
X_train = [x.title + ' ' + x.body for x in train.itertuples()]
y_train = [x.category for x in train.itertuples()]

test = pd.read_csv('data/ag_news_csv/test.csv', names=['category', 'title', 'body'])
X_test = [x.title + ' ' + x.body for x in test.itertuples()]
y_test = [x.category for x in test.itertuples()]

X_train_tfidf, dic, tfidf_model = to_tfidf(X_train)
X_test_tfidf, _, __ = to_tfidf(X_test, dic, tfidf_model)

The TF-IDF weighting used with the linear SVM achieves the accuracy of 91%. Because this is a multiclass classification problem, this metric is the same as hit@1, reported by StarSpace.

In [61]:
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_train)

svc = LinearSVC()
svc.fit(gensim.matutils.corpus2csc(X_train_tfidf, num_terms=len(dic)).T, le.transform(y_train))
y_predicted = svc.predict(gensim.matutils.corpus2csc(X_test_tfidf, num_terms=len(dic)).T)
print('Accuracy: {:.3f}'.format(metrics.accuracy_score(le.transform(y_test), y_predicted)))

Accuracy: 0.910


We have embeddings for a large number of words, so we can run clustering to see if the embeddings vectors can be used to partition words into four categories.

In [11]:
import numpy as np
from sklearn.cluster import KMeans

model = pd.read_csv('data/ag_news_csv/model.tsv', sep='\t', header=None, keep_default_na=False)
embeddings = model[model.columns[1:]]
kmeans = KMeans(n_clusters=4, random_state=42).fit(embeddings)

FileNotFoundError: [Errno 2] No such file or directory: 'data/ag_news_csv/model.tsv'

The first three produced clusters closely match the topics World, Sci/Tech, and Business, while the last and by far the largest cluster is less specific and contains words from all topics.

In [12]:
words_array = model[0].to_numpy()
for ci in range(kmeans.n_clusters):
    cluster_words = np.compress(kmeans.labels_==ci, words_array)
    print('Cluster {} ({} instances)'.format(ci, len(cluster_words)))
    print(cluster_words[:100])
    print('')

NameError: name 'model' is not defined